In [1]:
import pandas as pd
import numpy as np
import io
import os
import re
import sys
import tensorflow as tf
from pandasql import sqldf
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
root_path = 'C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\NLP_data\\goodbooks'
sys.path.append('C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\python_functions\\jl_nlp_pkg')

import nlpbasic.textClean as TextProcessing
import nlpbasic.docVectors as DocVector
import nlpbasic.dataExploration as dataExploration
import nlpbasic.lda as lda
import nlpbasic.tfidf as tfidf

pd.set_option('display.max_columns', 500)
print(tf.__version__)
pysqldf = lambda q: sqldf(q, globals())

C:\Users\luoyan011\.conda\envs\nlp\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


2.5.0


In [2]:
book_tags = pd.read_csv(os.path.join(root_path, "book_tags.csv"))
books = pd.read_csv(os.path.join(root_path, "books.csv"))
ratings = pd.read_csv(os.path.join(root_path, "ratings.csv"))
tags = pd.read_csv(os.path.join(root_path, "tags.csv"))
to_read = pd.read_csv(os.path.join(root_path, "to_read.csv"))

#load ratings latent variable dataset
user_latent = pd.read_csv(os.path.join(root_path, 'user_latent.csv'))
item_latent = pd.read_csv(os.path.join(root_path, 'item_latent.csv'))

In [3]:
def change_id_from_0(data, column):
    data_copy = data.copy()
    for i in column:
        data_copy[i] = data_copy[i] - 1
    return data_copy

book_tags = change_id_from_0(book_tags, ['goodreads_book_id'])
books = change_id_from_0(books, ['id'])
ratings = change_id_from_0(ratings, ['book_id','user_id'])
to_read = change_id_from_0(to_read, ['book_id','user_id'])
        
user_latent.columns = ['id','u_latent_1','u_latent_2','u_latent_3','u_latent_4',
                       'u_latent_5','u_latent_6','u_latent_7','u_latent_8',
                       'u_latent_9','u_latent_10','u_latent_11','u_latent_12',
                       'u_latent_13','u_latent_14','u_latent_15']
item_latent.columns = ['id','i_latent_1','i_latent_2','i_latent_3','i_latent_4',
                       'i_latent_5','i_latent_6','i_latent_7','i_latent_8',
                       'i_latent_9','i_latent_10','i_latent_11','i_latent_12',
                       'i_latent_13','i_latent_14','i_latent_15']

In [4]:
print('--------------book_tags--------------')
display(book_tags.head(3))
print('--------------books--------------')
display(books.head(3))
print('--------------ratings--------------')
display(ratings.head(3))
print('--------------tags--------------')
display(tags.head(3))
print('--------------to_read--------------')
display(to_read.head(3))
print('--------------user_latent--------------')
display(user_latent.head(3))
print('--------------item_latent--------------')
display(item_latent.head(3))

--------------book_tags--------------


,goodreads_book_id,tag_id,count
0,0,30574,167697
1,0,11305,37174
2,0,11557,34173


--------------books--------------


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,0,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,2,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


--------------ratings--------------


...

,user_id,book_id
0,0,111
1,0,234
2,0,532


--------------user_latent--------------


,id,u_latent_1,u_latent_2,u_latent_3,u_latent_4,u_latent_5,u_latent_6,u_latent_7,u_latent_8,u_latent_9,u_latent_10,u_latent_11,u_latent_12,u_latent_13,u_latent_14,u_latent_15
0,0,0.199595,-0.108096,0.247037,0.208254,0.106582,-0.123816,0.125920,0.384342,0.611112,-0.349587,-0.031930,-0.156987,0.041728,0.185200,-0.358743
1,1,-0.212097,-0.068822,0.102718,0.252274,-0.133377,-0.093753,-0.558407,0.492150,-0.254548,-0.076484,-0.131823,-0.259838,-0.156847,0.019408,-0.642446
2,2,-0.057435,-0.006853,0.118266,0.120967,-0.020337,-0.095097,-0.039628,0.067750,0.028180,0.038364,0.013028,-0.054933,-0.047005,0.054901,-0.032876


--------------item_latent--------------


,id,i_latent_1,i_latent_2,i_latent_3,i_latent_4,i_latent_5,i_latent_6,i_latent_7,i_latent_8,i_latent_9,i_latent_10,i_latent_11,i_latent_12,i_latent_13,i_latent_14,i_latent_15
0,0,1.900808,0.380494,0.661577,-0.665127,-0.355070,0.904718,0.856841,0.359961,-0.945763,-0.005923,0.378899,-0.479288,-0.702245,-0.751551,-0.541900
1,1,1.694283,0.602090,0.477443,-0.531324,-0.125256,0.900841,1.043496,0.370730,-1.422744,-0.050342,0.366465,-0.751425,-0.551353,-0.573139,-0.660848
2,2,1.265569,0.245617,0.385378,-0.438928,0.333491,0.216258,-0.090325,0.468572,-0.925843,-0.116226,0.675204,-0.455215,-0.768754,-0.879207,-0.654471


In [7]:
user_rating_counts = ratings[['user_id','book_id']].groupby('user_id').agg('count')
user_rating_counts.head(3)

,book_id
user_id,
0,3
1,3
2,2


In [9]:
to_read['rank'] = to_read.groupby('user_id')['book_id'].rank('dense')
to_read.head(10)

,user_id,book_id,rank
0,0,111,1.0
1,0,234,2.0
2,0,532,3.0
3,0,1197,4.0
4,0,1873,5.0
5,0,2057,6.0
6,0,3333,7.0
7,1,3,1.0
8,1,10,2.0
9,1,12,3.0


In [12]:
ratings['rank'] = ratings.groupby('user_id')['book_id'].rank('dense')
ratings.sort_values(by='user_id').head(10)

,book_id,user_id,rating,rank
117889,1179,0,4,1.0
488112,4892,0,3,2.0
625717,6284,0,4,3.0
796318,8033,1,4,1.0
875008,8854,1,5,2.0
959821,9761,1,4,3.0
890007,9013,2,1,1.0
893311,9048,2,1,2.0
326891,3272,3,2,1.0
837459,8463,3,5,3.0


In [21]:
query = """
select A.user_id
    , A.book_id
    , A.rating
    , B.original_title
    , B.authors
    , B.original_publication_year
    , B.language_code
    , B.average_rating
    , B.ratings_count
    , C.book_id as to_read_book_id
    , D.*
    , E.*
from ratings A
left join books B on A.book_id = B.id
left join to_read C on A.user_id = C.user_id and A.rank = C.rank
left join user_latent D on A.user_id = D.id
left join item_latent E on A.book_id = E.id
where B.language_code in ('eng','en-US','en-GB','en-CA')
"""
hybrid_data_base = pysqldf(query)
hybrid_data_base = hybrid_data_base.drop(columns = ['id'])
hybrid_data_base.head(3)


,user_id,book_id,rating,original_title,authors,original_publication_year,language_code,average_rating,ratings_count,to_read_book_id,u_latent_1,u_latent_2,u_latent_3,u_latent_4,u_latent_5,u_latent_6,u_latent_7,u_latent_8,u_latent_9,u_latent_10,u_latent_11,u_latent_12,u_latent_13,u_latent_14,u_latent_15,i_latent_1,i_latent_2,i_latent_3,i_latent_4,i_latent_5,i_latent_6,i_latent_7,i_latent_8,i_latent_9,i_latent_10,i_latent_11,i_latent_12,i_latent_13,i_latent_14,i_latent_15
0,313,0,5,The Hunger Games,Suzanne Collins,2008.0,eng,4.34,4780653,10.0,0.565416,0.546839,0.256609,-1.007740,-0.473202,0.049479,0.280110,0.210727,-0.532448,-0.860113,0.335041,0.148334,-0.337264,-0.586449,-0.603003,1.900808,0.380494,0.661577,-0.665127,-0.35507,0.904718,0.856841,0.359961,-0.945763,-0.005923,0.378899,-0.479288,-0.702245,-0.751551,-0.5419
1,438,0,3,The Hunger Games,Suzanne Collins,2008.0,eng,4.34,4780653,120.0,0.811270,0.483355,0.393725,-0.617435,0.300784,0.259413,0.422586,0.058931,-0.734391,-0.499431,-0.014451,0.281844,-0.162854,0.475232,-0.825033,1.900808,0.380494,0.661577,-0.665127,-0.35507,0.904718,0.856841,0.359961,-0.945763,-0.005923,0.378899,-0.479288,-0.702245,-0.751551,-0.5419
2,587,0,5,The Hunger Games,Suzanne Collins,2008.0,eng,4.34,4780653,2115.0,0.683785,0.593587,0.700573,-0.508013,-1.182165,-0.424068,0.672739,-0.196326,-0.031572,-0.659620,-0.111626,-0.663243,-0.828182,0.270393,-0.394489,1.900808,0.380494,0.661577,-0.665127,-0.35507,0.904718,0.856841,0.359961,-0.945763,-0.005923,0.378899,-0.479288,-0.702245,-0.751551,-0.5419


In [18]:
print(hybrid_data_base.shape)
print(ratings.shape)

(981756, 42)
(981756, 4)


In [22]:
hybrid_data_base.columns

Index(['user_id', 'book_id', 'rating', 'original_title', 'authors',
       'original_publication_year', 'language_code', 'average_rating',
       'ratings_count', 'to_read_book_id', 'u_latent_1', 'u_latent_2',
       'u_latent_3', 'u_latent_4', 'u_latent_5', 'u_latent_6', 'u_latent_7',
       'u_latent_8', 'u_latent_9', 'u_latent_10', 'u_latent_11', 'u_latent_12',
       'u_latent_13', 'u_latent_14', 'u_latent_15', 'i_latent_1', 'i_latent_2',
       'i_latent_3', 'i_latent_4', 'i_latent_5', 'i_latent_6', 'i_latent_7',
       'i_latent_8', 'i_latent_9', 'i_latent_10', 'i_latent_11', 'i_latent_12',
       'i_latent_13', 'i_latent_14', 'i_latent_15'],
      dtype='object')

## Data Embedding Processing

In [46]:
import tensorflow_hub as hub

In [ ]:
factor_columns = ['u_latent_{}'.format(i for i in range(15))] + ['i_latent_{}'.format(i for i in range(15))]
non_factor_columns = ['book_id', 'original_title', 'authors', 'original_publication_year',  
                      'rating', 'average_rating', 'ratings_count', 'to_read_book_id']
label_columns = 'to_read_book_id'

In [41]:
number_of_book_ids = len(hybrid_data_base.book_id.unique())
print('number_of_book_ids:', number_of_book_ids)
number_of_authors = len(hybrid_data_base.authors.unique())
print('number_of_authors:', number_of_authors)


number_of_book_ids: 10000
number_of_authors: 4664


In [38]:
def create_feature_columns(args):
    # create book_id feature column
    book_id_column = tf.feature_column.categorical_column_with_hash_bucket(
        key = 'book_id',
        hash_bucket_size = number_of_book_ids
    )
    # embed book_id into a lower dimensional representation
    embedded_book_columns = tf.feature_column.embedding_column(
        categorical_column = book_id_column,
        dimension = args['book_id_embedding_dimensions']
    )
    
    # create title feature column usng TF Hub
    embedded_title_column = hub.text_embedding_column(
        key = 'oriinal_title',
        module_spec = "https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2",
        trainable = False
    )
    
    #create author feature column
    author_column = tf.feature_column.categorical_column_with_hash_bucket(
        key = 'authors',
        hash_bucket_size = number_of_authors + 1
    )
    # embed author into a lower dimensional representative
    embedded_author_column = tf.feature_column.embedding_column(
        categorical_column = author_column,
        dimesion = args['author_embedding_dimensions']
    )
    
    #create original publication year boundaries list 
    original_publication_year_boundaries = [-500, 500, 1000, 1500, 1600, 1700, 1800, 1900, 1950, 
                                            1960, 1970, 1980, 1990, 2000, 2005, 2010, 2015]
    
    # create original_publication_year feature column using raw data
    original_publication_year_column = tf.feature_column.numeric_column(
        key = 'original_publication_year'
    )
    
    #create original_publication_year feature column using boundaries list
    original_publication_year_bucketized = tf.feature_column.bucketized_column(
        source_column = original_publication_year_column,
        boundaries = original_publication_year_boundaries
    )
    
    #create user and item factor feature columns from trained WALS model
    user_factors = [tf.feature_column.numeric_column(key = 'u_latent_' + str(i)) for i in range(15)]
    item_factors = [tf.feature_column.numeric_column(key = 'i_latent_' + str(i)) for i in range(15)]
    rating_columns = tf.feature_column.numeric_column(key = 'rating')
    average_rating_columns = tf.feature_column.numeric_column(key = 'average_rating')
    ratings_count_columns = tf.feature_column.numeric_column(key = 'ratings_count')

    feature_columns = [
        embedded_book_columns,
        embedded_title_column,
        embedded_author_column,
        original_publication_year_bucketized,
        rating_columns,
        average_rating_columns,
        ratings_count_columns
    ] + user_factors + item_factors 
    return feature_columns

In [49]:
feature_para = {
    'author_embedding_dimensions' : 500,
    'book_id_embedding_dimensions' : 1000
}
create_feature_columns(feature_para)

NotFoundError: NewRandomAccessFile failed to Create/Open: C:\Users\LUOYAN~1\AppData\Local\Temp\tfhub_modules\89372f84accb892aa8bf438f7762756cfc715013\tfhub_module.pb : The system cannot find the file specified.
; No such file or directory